In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
#defining path
#path='/data/nkulkarni/Kaggle/Heart_Failure_Prediction'

#importing data
#master_df=pd.read_csv(path+'/Data/heart_failure_clinical_records_dataset.csv')
master_df=pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')


In [ ]:
#Adding a key identifier to the data in the begining - every row is a unique case - 0 is the position indication insert column to the front
master_df.insert(0,'key','') #creates a bank column 
master_df['key'] = range(1, len(master_df.index)+1) #fill the column with sequencial number

In [ ]:
master_df_cont=master_df[['key', 'age', 'anaemia','diabetes','high_blood_pressure','sex','smoking','time','serum_sodium','platelets','serum_creatinine','creatinine_phosphokinase','ejection_fraction','DEATH_EVENT']]

In [ ]:
features2=['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking']

In [ ]:
#choose the dataset (using continous for tree based algo)
model_data=master_df_cont

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(model_data[features2], model_data['DEATH_EVENT'], test_size=0.25, random_state=4,
                                                   #stratify = model_data[strat_cols_asis+strat_cols_constructed]
                                                   )

#creating a flag to indicate if a row is in training dataset or test dataset
model_data['is_test'] = ~model_data.index.isin(X_train.index)

In [ ]:
X_train.head()

In [ ]:
#building base model using Logistic Regression CV algorithm
#try with class_weight=balanced, deafult is none 
#try with warm_start=True, default is false - didnt have any difference (usless for liblinear solver)

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report

LR_clf = LogisticRegression(random_state=4).fit(X_train, y_train)
y_pred=LR_clf.predict(X_test)
y_pred_proba=LR_clf.predict_proba(X_test) #showing the probablity of getting death_event=0 and 1 for every record in test data
y_pred_df=LR_clf.decision_function(X_test)
fscore=precision_recall_fscore_support(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

# Model Evaluation - https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
print('Train Score: ',LR_clf.score(X_train, y_train))
print('Test Score: ',LR_clf.score(X_test, y_test))
#print('Probability estimates : ',y_pred_proba)
#print('Confidence scores : ', y_pred_df )
#print('precision_recall_fscore_support', fscore)
print('Confusion Matrix: \n', cm)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
#print('classification report for Logistic Regression\n', classification_report(y_test, y_pred))

#ROC curve. See AUC to see if the model is useful - AUC score 1 represents perfect classifier, and 0.5 represents a worthless classifier.
y_pred_proba = LR_clf.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()
print(auc)

#Feature importance 
feature_importance=pd.DataFrame(np.hstack((np.array([features2]).T, LR_clf.coef_.T)), columns=['feature', 'importance'])
feature_importance['importance']=pd.to_numeric(feature_importance['importance']).abs()
feature_importance.sort_values(by='importance', ascending=False)